## The Trade Lifecycle

In [1]:
import os

os.chdir("..")

### Portfolio Setup

In [4]:
from pytrade.compliance import Compliance
from pytrade.broker import Broker
from pytrade.assets import (
    reset,
    Cash,
    Stock,
    Portfolio,
)

Let's initialise our portfolio.

In [5]:
reset()
usd = Cash("USD")
stock = Stock("XXX US", 10, currency_code="USD")
portfolio = Portfolio("USD")

portfolio.transfer(usd, 1000)
portfolio.transfer(stock, 100)

print(portfolio)

Portfolio('USD'):
Cash('USD', 1.0, currency_code='USD'): 1,000
Stock('XXX US', 10, currency_code='USD'): 100


In [6]:
portfolio.value

2000.0

### Trade Objects
Each trade needs the an associated portfolio, asset and the number of units we wish to trade.

In [8]:
from pytrade import Trade

trade = Trade(portfolio, stock, 100)
print(trade)

Trade(Portfolio('USD'), 'XXX US', 100)


### The Trade Lifecycle
Each trade is linked to a portfolio object which itself has associated compliance rules and executing brokers. 

In [10]:
portfolio.compliance

In [11]:
portfolio.broker

By default portfolios start with an empty compliance module (one that contains no compliance rules) and a broker that executes trades at the last available price with no fees. These options are configurable and we'll touch on them later. However, the basic trade pipeline is as follows:

-  A trade is created / proposed with an associated portfolio.


-  Compliance is run on this trade to check whether it passes any rules including (but not limited to) position limits and restricted securities.


-  If compliance passes for this trade then it is sent to the portfolio's broker object for execution. This may incur some fees and slippage depending on how the broker object is defined.

The trade pipeline does this in the background for you.

In [22]:
from pytrade.trade import trade_pipeline

trade_pipeline

Our pipeline starts with the compliance handler at its head and ends with trade execution (if compliance passes).

## Compliance
We can initialise a new compliance module as follows.

In [12]:
from pytrade.compliance import Compliance

compliance = Compliance()
compliance

In [14]:
len(compliance)  # there are currently zero rules

0

Once we have a new compliance object we can begin adding rules with the add_rule method.

In [15]:
from pytrade.compliance import UnitLimit

compliance.add_rule(
    UnitLimit(stock, 100)
)

len(compliance)

1

If we wanted to add a unit limit to our portfolio we can write.

In [16]:
portfolio.compliance = Compliance().add_rule(
    UnitLimit(stock, 1000)
)

All trades start with an attribute of passed_compliance = False. A trade has to pass all compliance rules to pass compliance as a whole.

In [18]:
trade.passed_compliance

False

To create custom compliance rules you'll have to inherit from the ComplianceRule base class and implement the passes method (which takes the portfolio as an argument and returns a boolean value).

In [25]:
from pytrade.compliance import ComplianceRule


class RestrictedSecurityCode(ComplianceRule):
    def __init__(self, restricted_code):
        super().__init__()
        self._restricted_code = restricted_code

    def passes(self, portfolio):
        position = portfolio.get_holding_units(self._restricted_code)
        if abs(position) != 0:
            return False
        return True


In [26]:
Compliance().add_rule(
    RestrictedSecurityCode("ZZB AU")
).passes(portfolio)

True

In [27]:
Compliance().add_rule(
    RestrictedSecurityCode("XXX US")
).passes(portfolio)

False

In this second case compliance fails as the portfolio holds our restricted security code. Unit and weight limits are defined in the compliance module.

In [28]:
from pytrade.compliance import UnitLimit, WeightLimit

In [29]:
issubclass(UnitLimit, ComplianceRule), issubclass(WeightLimit, ComplianceRule)

(True, True)

## Broker Execution
As mentioned earlier the default broker strategy is to execute trades (that pass compliance) fully at the last available price and with no fee.

In [31]:
portfolio.broker

In [33]:
[item for item in dir(portfolio.broker) if not item.startswith("__")]

['_charges_strategy', '_execution_strategy', 'execute']

From the object dictionary we can see that brokers have both a charges strategy and an execution strategy. We can configure portfolio brokers with these options.

In [35]:
from pytrade.broker import (  # execution strategies
    FillAtLast,  # default
    FillAtLastWithSlippage,
)

from pytrade.broker import (  # charge strategies
    NoCharges,  # default
    FixedRatePlusPercentage,
)

In [37]:
print(portfolio)

Portfolio('USD'):
Cash('USD', 1.0, currency_code='USD'): 1,000
Stock('XXX US', 10, currency_code='USD'): 100


In [38]:
print(trade)

Trade(Portfolio('USD'), 'XXX US', 100)


In [39]:
print(stock)

Stock('XXX US', 10, currency_code='USD')


Suppose we redefine the portfolio broker.

In [40]:
portfolio.broker = Broker(
    execution_strategy=FillAtLastWithSlippage(0.01),
    charges_strategy=FixedRatePlusPercentage(20, 0.01, currency_code="USD"),
)

If the broker executes our trade there will be 1% slippage, a USD 20 fixed charge and a 1% variable charge.

The trade value is 100 shares * USD 10 price = USD 1000, so 1% of this is USD 10.
*  USD 10 slippage
*  USD 20 fixed broker charge
*  USD 10 variable broker charge

In [42]:
portfolio.broker.execute(trade)

In [43]:
print(portfolio)

Portfolio('USD'):
Cash('USD', 1.0, currency_code='USD'): -40
Stock('XXX US', 10, currency_code='USD'): 200


There was cash of USD 1000 in the portfolio to start with. This would have been the cost of buying 100 shares had there been no slippage or broker charges. However, after the trade we end up short USD by the amount of our total charges = USD 40. Our stock position has also gone up by 100 shares.

***